# 自己用tensorflow2改写的原来的代码

In [2]:
import tensorflow as tf
import numpy as np

# 手动实现数据预处理
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = tf.cast(train_images[..., tf.newaxis]/255.0, tf.float32)  # 添加通道维度并归一化
test_images = tf.cast(test_images[..., tf.newaxis]/255.0, tf.float32)
train_labels = tf.one_hot(train_labels, 10)  # 手动one-hot编码
test_labels = tf.one_hot(test_labels, 10)

# 初始化参数
def weight_variable(shape):
    return tf.Variable(tf.random.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

# 卷积和池化操作
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool2d(x, ksize=2, strides=2, padding='SAME')

# 构建网络
class ManualCNN(tf.Module):
    def __init__(self):
        super().__init__()
        # 卷积层1
        self.W_conv1 = weight_variable([7,7,1,32])
        self.b_conv1 = bias_variable([32])
        
        # 卷积层2
        self.W_conv2 = weight_variable([5,5,32,64])
        self.b_conv2 = bias_variable([64])
        
        # 全连接层
        self.W_fc1 = weight_variable([7 * 7 * 64, 1024])
        self.b_fc1 = bias_variable([1024])
        self.W_fc2 = weight_variable([1024,10])
        self.b_fc2 = bias_variable([10])
    
    def __call__(self, x, training=False):
        # 输入重塑
        x = tf.reshape(x, [-1,28,28,1])
        
        # 卷积层1 + ReLU + 池化
        h_conv1 = tf.nn.relu(conv2d(x, self.W_conv1) + self.b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)
        
        # 卷积层2 + ReLU + 池化
        h_conv2 = tf.nn.relu(conv2d(h_pool1, self.W_conv2) + self.b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        
        # 全连接层
        h_pool2_flat = tf.reshape(h_pool2, [-1,7 * 7 * 64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, self.W_fc1) + self.b_fc1)
        
        # 训练时应用Dropout
        if training:
            h_fc1 = tf.nn.dropout(h_fc1, rate=0.3)
        
        return tf.nn.softmax(tf.matmul(h_fc1, self.W_fc2) + self.b_fc2)

# 实例化模型和优化器
model = ManualCNN()
optimizer = tf.optimizers.Adam(1e-4)

# 手动实现训练循环
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        # 手动计算交叉熵损失
        loss = -tf.reduce_mean(tf.reduce_sum(labels * tf.math.log(predictions), axis=1))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# 计算准确率
def compute_accuracy(images, labels):
    predictions = model(images)
    correct = tf.equal(tf.argmax(predictions,1), tf.argmax(labels,1))
    return tf.reduce_mean(tf.cast(correct, tf.float32))

# 训练过程
for epoch in range(2000):
    # 生成batch
    idx = np.random.choice(len(train_images), 100)
    batch_images = tf.gather(train_images, idx)
    batch_labels = tf.gather(train_labels, idx)
    
    loss = train_step(batch_images, batch_labels)
    
    if epoch % 100 == 0:
        acc = compute_accuracy(test_images[:1000], test_labels[:1000])
        print(f"Epoch {epoch}: Loss={loss:.4f}, Accuracy={acc:.4f}")

Epoch 0: Loss=12.6538, Accuracy=0.1210
Epoch 100: Loss=0.8535, Accuracy=0.8760
Epoch 200: Loss=0.2675, Accuracy=0.9230
Epoch 300: Loss=0.2074, Accuracy=0.9370
Epoch 400: Loss=0.2455, Accuracy=0.9480
Epoch 500: Loss=0.2204, Accuracy=0.9520
Epoch 600: Loss=0.0917, Accuracy=0.9640
Epoch 700: Loss=0.1634, Accuracy=0.9660
Epoch 800: Loss=0.1870, Accuracy=0.9700
Epoch 900: Loss=0.0507, Accuracy=0.9690
Epoch 1000: Loss=0.1832, Accuracy=0.9760
Epoch 1100: Loss=0.0924, Accuracy=0.9750
Epoch 1200: Loss=0.0619, Accuracy=0.9770
Epoch 1300: Loss=0.0321, Accuracy=0.9740
Epoch 1400: Loss=0.0544, Accuracy=0.9780
Epoch 1500: Loss=0.0831, Accuracy=0.9730
Epoch 1600: Loss=0.1264, Accuracy=0.9780
Epoch 1700: Loss=0.0489, Accuracy=0.9790
Epoch 1800: Loss=0.0867, Accuracy=0.9820
Epoch 1900: Loss=0.0979, Accuracy=0.9820
